In [1]:
# Import the necessary libraries

import pandas as pd
from bs4 import BeautifulSoup
import re
import requests

### Web scraping

In [2]:
# Create an empty list to store listings

listings = []

# Scrap data for 1000 rows
for i in range(1, 49):
    url = f"https://nigeriapropertycentre.com/for-rent?page={i}"
    header_code = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=header_code)

    soup = BeautifulSoup(response.content, 'html.parser')
    info = soup.find_all('div', class_='wp-block-body')

    for data in info:
        description = data.find('h4', class_='content-title').get_text()
        location = data.find('address', class_='voffset-bottom-10').get_text()
        rent_value = data.find('span', class_='pull-sm-left').get_text()
        prices = data.find_all('span', class_='price')
        price = prices[1].get_text()
        
        # Append the data to the listings list as a dictionary
        listings.append([description, location, rent_value, price])

In [3]:
# Assign the scraped data to a dataframe

property_data = pd.DataFrame(listings, columns = ['Description', 'Location', 'Rent_value', 'Price'])

In [4]:
# Display a sample of the data

property_data.sample()

,Description,Location,Rent_value,Price
279,3 bedroom flat / apartment for rent,"Banana Island, Ikoyi, Lagos","\n₦20,000,000 per annum","20,000,000"


In [5]:
# Extract City and State from the Location Column

property_data['City'] = property_data['Location'].str.split(',').str[-2].str.strip()
property_data['State'] = property_data['Location'].str.split(',').str[-1].str.strip()
property_data.head(5)

,Description,Location,Rent_value,Price,City,State
0,Conference / meeting / training room for rent,"Kumasi Crescent, Wuse 2, Abuja","\n₦80,000 per day","80,000",Wuse 2,Abuja
1,2 bedroom flat / apartment for rent,"Ikate, Ikate Elegushi, Lekki, Lagos","\n₦2,300,000 per annum","2,300,000",Lekki,Lagos
2,4 bedroom semi-detached duplex for rent,"Ikota, Lekki, Lagos","\n₦4,000,000 per annum","4,000,000",Lekki,Lagos
3,5 bedroom detached duplex for rent,"VGC, Lekki, Lagos","\n₦7,000,000 per annum","7,000,000",Lekki,Lagos
4,Office space for rent,"Ologolo, Lekki, Lagos","\n₦1,500,000 per annum","1,500,000",Lekki,Lagos


In [6]:
# Check the number of rows and columns

row = property_data.shape[0]
column = property_data.shape[1]

print(f'This property data has {row} rows and {column} columns')

This property data has 1008 rows and 6 columns


In [7]:
# Check for null values in the dataset

property_data.isna().sum()

Description    0
Location       0
Rent_value     0
Price          0
City           0
State          0
dtype: int64

In [8]:
property_data.dtypes

Description    object
Location       object
Rent_value     object
Price          object
City           object
State          object
dtype: object

In [9]:
# Change the Price column to a float datatype to allow for math operatons

property_data['Price'] = property_data['Price'].str.replace(',', '').astype(float)

In [10]:
# Use the 5 point summary to check the dataset

pd.options.display.float_format = '{:.0f}'.format      # Set the float format to display without exponential notation
property_data.describe()

,Price
count,1008
mean,11187421
std,84400619
min,250
25%,2000000
50%,4000000
75%,8000000
max,2160000000


#### Derive Insights from the dataset

In [11]:
max_prop = property_data['Price'].max()
min_prop = property_data['Price'].min()

print(f'The most and least expensive properties cost {max_prop} Naira and {min_prop} Naira to rent respectively ')

The most and least expensive properties cost 2160000000.0 Naira and 250.0 Naira to rent respectively 


In [12]:
pop_state = property_data['State'].mode()[0]
print(f'{pop_state} State has the largest number of properties to be rented')

Lagos State has the largest number of properties to be rented


In [13]:
highest_prop = property_data[property_data['Price'] == property_data.Price.max()]

print(f'The most expensive property is a {highest_prop.Description} located in {highest_prop.City} area of {highest_prop.State} State\n and it costs {highest_prop.Price} Naira to rent')

The most expensive property is a 107    Tank farm for rent
Name: Description, dtype: object located in 107    Apapa
Name: City, dtype: object area of 107    Lagos
Name: State, dtype: object State
 and it costs 107   2160000000
Name: Price, dtype: float64 Naira to rent


In [14]:
#Calculate the average rent costs per State

state_avg_price = property_data.groupby('State')['Price'].mean().reset_index(name='Average_rent')
state_avg_price

,State,Average_rent
0,Abuja,7203794
1,Lagos,12021157
2,Ogun,506667
3,Oyo,2500000
4,Rivers,4000000


In [15]:
#Calculate the average rent costs per City

city_avg_price = property_data.groupby('City')['Price'].mean()
city_avg_price

City
Agege                     3200000
Ajah                      1740909
Apapa                  1710000000
Apo                       4070000
Arepo                      900000
Asokoro District         24142857
Ayobo                      450000
Bwari                     1200000
Durumi                    2000000
Egbe                      4500000
Gaduwa                    3916667
Galadimawa                4629000
Garki                     3500000
Gbagada                   2911111
Guzape District          12750000
Gwarinpa                  3272727
Ibadan                    2500000
Ibeju Lekki               2362500
Idu Industrial              25000
Ifako-Ijaiye               250000
Ikeja                     8605146
Ikorodu                    466667
Ikotun                    1000000
Ikoyi                    19276933
Ilupeju                    600000
Ipaja                      990000
Isheri North              1866667
Isolo                    25400000
Jabi                     10525000
Jahi     

In [16]:
#Display the number of properties for rent in each city and also display the states

num_properties = property_data.groupby('State')['City'].value_counts().sort_values(ascending = False)
num_properties

State   City                
Lagos   Lekki                   455
        Ajah                     88
        Victoria Island (VI)     84
        Ikoyi                    75
        Ikeja                    36
Abuja   Jahi                     31
Lagos   Yaba                     27
Abuja   Maitama District         17
        Wuse 2                   14
        Life Camp                13
        Wuye                     12
        Gwarinpa                 11
Lagos   Ipaja                    10
Abuja   Katampe                   9
Lagos   Magodo                    9
        Gbagada                   9
        Ibeju Lekki               8
Abuja   Asokoro District          7
Lagos   Shomolu                   6
        Surulere                  5
        Ketu                      5
Abuja   Apo                       5
        Utako                     5
        Mabushi                   5
        Lugbe District            4
        Jabi                      4
Lagos   Ogudu                     3